In [1]:
import torch
import pandas  as pd

from lime.lime_text import LimeTextExplainer
import os
import pickle
import eli5

from transformers import T5ForConditionalGeneration, RobertaTokenizer, RobertaForSequenceClassification
import torch
import pandas as pd
import torch.nn.functional as F
import torch.nn as nn

import numpy as np
from eli5.lime import TextExplainer
from eli5.lime.samplers import MaskingTextSampler, MaskingTextSamplers


In [2]:
class CodeBert(nn.Module):
    def __init__(self,drop_prob=0.5):
        super(CodeBert, self).__init__()
 
        self.bert=RobertaForSequenceClassification.from_pretrained("microsoft/codebert-base")
        for param in self.bert.parameters():
            param.requires_grad = True
        
        # dropout layer
        self.dropout = nn.Dropout(drop_prob)
           
          
        self.sig = nn.Sigmoid()
 
    def forward(self, x):
        batch_size = x.size(0)
        #生成bert字向量
        x=self.bert(x)[0]     #bert 字向量
        
        # dropout and fully-connected layer
        out = self.dropout(x)
        out = self.sig(out)
        
        return out

In [4]:
def predict_example(test_comment_list):

    USE_CUDA = torch.cuda.is_available()

    net = CodeBert()
    net.load_state_dict(torch.load(model_path))
    if(USE_CUDA):
        net.cuda()
    # tokenize
    tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
    result_comments_id = tokenizer(test_comment_list,
                                    padding=True,
                                    truncation=True,
                                    max_length=512,
                                    return_tensors='pt')
    tokenizer_id = result_comments_id['input_ids']
    inputs = tokenizer_id
    batch_size = inputs.size(0)
    # initialize hidden state

    if(USE_CUDA):
        inputs = inputs.cuda()

    net.eval()
    with torch.no_grad():
        # get the output from the model
        output= net(inputs)
        output=torch.nn.Softmax(dim=1)(output)
        return output.cpu().detach().numpy()
        

In [5]:
batch_size = 32
model_path = "../models/bert_model"
bert_model_class = "microsoft/codebert-base"
test_dataset_path = '../data/clean_data/test.csv'

label_names = ['negative', 'positive']

In [6]:
explainer = TextExplainer(n_samples = 50, random_state=2022, sampler=MaskingTextSampler())

In [12]:
# model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-base")
# model_path = "D:/2022summerproject/models/pytorch_model.bin"
# model.load_state_dict(torch.load(model_path))

# tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-base")

test = pd.read_csv(",,/data/clean_data/test.csv")
k=11
example0 = test.iloc[k*2]['new_code']
example1 = test.iloc[k*2+1]['new_code']


In [30]:
# def prediction(example_list):
#     logits=[]
#     for example in example_list:
#         input_ids = tokenizer(example, truncation=True, return_tensors="pt").input_ids
#         generated_ids = model.generate(input_ids, max_length=512, output_scores=True, return_dict_in_generate=True)
#         logit = generated_ids.scores[1][0][20:22]
#         logit = F.softmax(logit, -1)
#         p = logit.tolist()[0]
#         logit = [p,1-p]
#         logits.append(logit)
#     return np.array(logits)

In [15]:
explainer.fit(example0, predict_example)
explainer.show_prediction(target_names=['negative', 'positive'])

Some weights of the model checkpoint at microsoft/codebert-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be 

In [19]:
explainer.fit(example1, predict_example)
explainer.show_prediction(target_names=['negative', 'positive'])

Some weights of the model checkpoint at microsoft/codebert-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be 